# pytket-dqc Example Usage

In this notebook we gather some example uses of the pytket-dqc package.

## Networks

Near term networks of quantum servers are specified by two properties. The fist is the server coupling, detailing which servers are connected to which others. This is specified by a list of pairs of integers, where each pair signifies that there is a connection between those two servers. The second is the qubits each server contains. This is specified by a dictionary from the server to a list of qubits it contains. 

An example is given below, where blue lines indicate connections between servers, and red lines indicate connections between qubits within servers. The vertex labels are the indexes of the qubits.

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[0,1,2,3], 1:[4,5,6], 2:[7,8,9,10,11], 3:[12,13]})
network.draw_nisq_network()

## Distributed Circuits

The `DistributedCircuits` class adds some additional properties and methods to the standard `Circuit` pytket class. These predominantly relate to a hypergraph representation of the circuit. `DistributedCircuits` are initialised from a standard tket `Circuit` object, as seen in the following example. Additional functionality is provided to visualise the corresponding hypergraph. For details on the correspondence between circuits and hypergraphs please see the paper [Automated distribution of quantum circuits via hypergraph partitioning](https://arxiv.org/abs/1811.10972).

In [ ]:
from pytket_dqc.circuits import DistributedCircuit
from pytket.circuit.display import render_circuit_jupyter
from pytket import Circuit

circ = Circuit(4).CZ(0,3).Rz(0.5,3).CZ(1,3).CZ(2,3).Rz(0.5,3)

dist_circ = DistributedCircuit(circ)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

The accepted gate set for pytket-dqc is at present Rx, CZ, CX, Rz, and Measure. Note that in the case of the hyperedges that relate to CX, hyperedges are weighted. This is because they cannot be acted between servers in some directions using only one e-bit, and 2 may be required to perform a teleportation operation.

In [ ]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter

circ = Circuit(2)
circ.CZ(0,1)
circ.Rz(0.3,1)
circ.CX(1,0)

dist_circ = DistributedCircuit(circ)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

pytket-dqc includes some utilities for rebasing your circuit if it is not in the correct gate set initially.

In [ ]:
from pytket_dqc.utils import dqc_rebase

circ = Circuit(3).CY(0,1).CZ(1,2).H(1).CX(1,0)
render_circuit_jupyter(circ)

dqc_rebase.apply(circ)
render_circuit_jupyter(circ)

## Distributors

pytket-dqc provides several distributors, which are themselves a selection of methods to assign qubits and gates to servers. Their aim is to return a placement which minimises the e-bit cost of the resulting implementation.

Here we will work though some example use of these distributors. To do so let's define a simple network.

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2]], {0:[0], 1:[1,2], 2:[3,4]})
network.draw_nisq_network()

Let's also define a circuit to distribute. Some classes of circuits are predefined within pytket-dqc. Cyclic circuits, where CZ gates act in a circle, are one such class of circuits. These circuits are defined by the number of qubits they act on and the number of layers of cycles.

In [ ]:
from pytket_dqc.circuits import CyclicDistributedCircuit

dist_circ = CyclicDistributedCircuit(4,2)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

One such is Brute, which performs a brutefoce search of all placements of qubits and gates onto servers, returning the one with the lowest cost. It is the slowest method, but returns the best result every time.

In [ ]:
from pytket_dqc.distributors import Brute
import time

distributor = Brute()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

Annealing is another approach, which uses simulated annealing as a means to arrive at a valid placement.

In [ ]:
from pytket_dqc.distributors import Annealing

distributor = Annealing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

GraphPartitioning uses the [Karlsruhe Hypergraph Partitioning Framework](https://kahypar.org/) to derive a placement.

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))
assert placement.is_valid(dist_circ, network)

The current version of GraphPartitioning is guaranteed to output a valid placement.

Routing makes use of routing and placement and routing techniques available in TKET. Here the network architecture as a whole is interpreted as a backend architecture, with noise on edges between servers set to be very high. Routing is guaranteed to generate a valid placement, and often very quickly. Unfortunately, it does not do a great job of distinguishing between connections within servers and connections between them. This can result in a high e-bit cost. Routing will also alter the circuit

In [ ]:
from pytket_dqc.distributors import Routing

distributor = Routing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

## Larger Example

Let's see how far we can push these schemes. Let's create a larger network:

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[i for i in range(10)], 1:[i for i in range(10, 20)], 2:[i for i in range(20,30)], 3:[i for i in range(30,40)]})
network.draw_nisq_network()

and a larger circuit:

In [ ]:
from pytket_dqc.circuits import CyclicDistributedCircuit
from pytket.circuit.display import render_circuit_jupyter

dist_circ = CyclicDistributedCircuit(35,2)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

We have already seen that Brute can be quite slow, so let's not use that here. Let's see how the others perform, starting with Annealing. In this case we see that it performs the slowest of the three schemes. However it does not take an unreasonably long time, and the solution is reasonably good.

In [ ]:
from pytket_dqc.distributors import Annealing
import time

distributor = Annealing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

GraphPartitioning is the quickest, and produces the best result. In this case this is not unsurprising. The implementation of kahypar is highly optimised and so the result is arrived at very quickly.

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

The placement cost of Routing is the worst, although this is not unreasonably so. Again it arrives a the solution very quickly.

In [ ]:
from pytket_dqc.distributors import Routing

distributor = Routing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

Let's finally take a more complex network:

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [2,3]], {0:[i for i in range(5)], 1:[i for i in range(5, 20)], 2:[i for i in range(20,30)], 3:[i for i in range(30,45)]})
network.draw_nisq_network()

and circuit:

In [ ]:
from pytket_dqc.circuits import RegularGraphDistributedCircuit
from pytket.circuit.display import render_circuit_jupyter

dist_circ = RegularGraphDistributedCircuit(34,3,5)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

Annealing now takes a non-negligible time to find a placement. However, the solution is valid, and of those that generate a valid solution, it is the best

In [ ]:
from pytket_dqc.distributors import Annealing
import time

distributor = Annealing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

In the case of GraphPartitioning we can choose how many refinement rounds are done after the initial partitioning by KaHyPar. These refinement rounds are meant to reduce the cost taking into account the restricted network connectivity, which KaHyPar cannot manage on its own. If `num_rounds` is set to zero, the outcome corresponds to that of KaHyPar with no refinement.

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network, num_rounds=0)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

Alternatively, we can ask for `num_rounds=100` so that there is some refinement being done. This improves the placement cost significantly at the cost of some extra time.

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network, num_rounds=100)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

If we do not specify a value for `num_rounds` it defaults to 1000 rounds. The algorithm has a halting criterion that stops it before running all of those rounds if it realises that the improvement being made on each iteration is no longer noticeable.

Routing is now noticeably slow, and the solution quite poor.

In [ ]:
from pytket_dqc.distributors import Routing

distributor = Routing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

## Circuit Generation

Once a placement has been obtained, it it possible to use pytket-dqc to generate a distributed circuit which implements the original circuit between several servers. There are two circuit generation methods which may be used according to the detail required. 

Let's consider the following simple circuit as an example.

In [ ]:
from pytket_dqc.circuits import DistributedCircuit
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

circ = Circuit(2)
circ.CZ(0,1).Rx(0.3,0).CZ(0,1)
render_circuit_jupyter(circ)

dist_circ = DistributedCircuit(circ)
dist_circ.draw()

Which we will place on the following simple network

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2]], {0:[0], 1:[1], 2:[2]})
network.draw_nisq_network()

The first circuit generation method will simply relabel the qubit registers.

In [ ]:
from pytket_dqc.placement import Placement

placement = Placement({0:1, 1:2, 2:0, 3:0})
assert dist_circ.is_placement(placement)

circ_with_dist = dist_circ.to_relabeled_registers(placement)
render_circuit_jupyter(circ_with_dist)

The second will additionally generate all of the necessary distributed gates.

In [ ]:
circ_with_dist = dist_circ.to_pytket_circuit(placement, network)
render_circuit_jupyter(circ_with_dist)

Notice firstly that there is one link qubit added for each e-bit. Secondly we see that the gates are acted on a different server from the servers where the qubits are placed. As such in this case we have one server which consists of only link qubits.

# Small example to check trivial partition

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1]], {0:[i for i in range(3)], 1:[i for i in range(3, 9)]})
network.draw_nisq_network()

In [ ]:
from pytket.circuit import Circuit, CircBox, QControlBox, Op, OpType
from pytket_dqc.circuits import DistributedCircuit
from pytket.circuit.display import render_circuit_jupyter

op = Op.create(OpType.V)
cv = QControlBox(op, 1)

circ = Circuit(4)
circ.CZ(0,1) 
circ.CZ(0,2) 
circ.Rx(0.3,2)
circ.CZ(1,3) 
circ.Rx(0.3,0)
circ.CZ(1,2) 
circ.CZ(0,3) 
dist_circ = DistributedCircuit(circ)

render_circuit_jupyter(circ)

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()


placement = distributor.distribute(dist_circ, network)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

The final placement cost is zero since all of the vertices are placed in the same server. Such behaviour is guaranteed if `num_rounds` is left to its default value or is greater than 0.

# Working with cache

The current implementation of GraphPartitioning (and, in particular, the refinement algorithm) makes use of a cache to store precomputed costs. For each subset of servers, the cache stores the cost of communicating these servers in an efficient way (finding their Steiner tree in the network). The size of this cache can be toggled using parameter `cache_limit` which establishes a limit on the cardinality of the subsets of servers whose cost will be stored in cache: if the subset has more servers than `cache_limit` then its cost will be computed but not stored in the cache. If there are N servers and m = `max_key_size` then the cache will store up to N^m values. Default value is 5.

Let's test the effect of the cache on performance for a large example.

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [2,3]], {0:[i for i in range(5)], 1:[i for i in range(5, 20)], 2:[i for i in range(20,30)], 3:[i for i in range(30,45)]})
network.draw_nisq_network()

In [ ]:
from pytket_dqc.circuits import RegularGraphDistributedCircuit
from pytket.circuit.display import render_circuit_jupyter

dist_circ = RegularGraphDistributedCircuit(34,3,5)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

Using `cache_limit=5` (which is the default value) we get:

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network, seed=1, cache_limit=5)
print("time to distribute", time.time() - start)
print("final placement cost", placement.cost(dist_circ, network))

And if set `cache_limit=0` the cache is never used:

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network, seed=1, cache_limit=0)
print("time to distribute", time.time() - start)
print("final placement cost", placement.cost(dist_circ, network))

The final placement cost should not be affected (if it is, it would be due to the random choices within the algorithm). The point of using a cache is to make a difference in time, but we see that its effect is unnoticeable. This is likely because the network in this example is simple enough that calculating the Steiner tree takes virtually no time. It is likely that in general this will be the case and we would rather set `cache_limit=0` by default (requires testing on more complex networks and larger circuits).